In [14]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns

In [25]:
# Create a random distribution and plot

arr = np.random.normal(loc=0.0,size=10)
arr

array([-0.09442886, -0.2580902 , -0.62883332,  0.6054966 , -1.02353233,
        1.72691258, -0.83458674, -1.21474086, -0.32098559,  0.27655918])